In [1]:
clientResultsFilename = 'client-results.txt'
serverResultsFilename = 'server-results.txt'
paramsFilename = 'params.txt'
parsedResultsFilename = 'parsed-results.csv'

print(clientResultsFilename, serverResultsFilename, paramsFilename, parsedResultsFilename)

client-results.txt server-results.txt params.txt parsed-results.csv


In [2]:
clientResultsFile = open(clientResultsFilename, "r")

# Search in client-results
currentRun = None
netIO = None
bandwidth = None
replyTime = None
results = dict()
for line in clientResultsFile.readlines():
    line = line.rstrip()
    if currentRun == None and line.startswith("run"):
        currentRun = line.split(" ")[1]
    elif line.startswith("Net I/O"):
        netIO = line.split(": ")[1].split("KB/s")[0]
    elif line.startswith("[  3]") and line.endswith("Mbits/sec"):
        bw = line.split("   ")[2].split(" ")[0]
        bandwidth = str(float(bw) * 125)
    elif line.startswith("Reply time [ms]"):
        replyTime = line.split(": ")[1].split(" ")[1]
    elif line.startswith("Connection time [ms]: connect"):
        connectionTime = line.split(": ")[1].split(" ")[1]
    elif currentRun != None and line.startswith("run"):
        results[currentRun] = { "connectionTime": connectionTime, "replyTime": replyTime, "netIO": netIO, "bandwidth": bandwidth }
        currentRun = line.split(" ")[1]

results[currentRun] = { "connectionTime": connectionTime, "replyTime": replyTime, "netIO": netIO, "bandwidth": bandwidth }
# print(results)

In [3]:
serverResultsFile = open(serverResultsFilename, "r")

currentServerRun = None
utilIsNextLine = False
util = None
for line in serverResultsFile.readlines():
    line = line.rstrip()
    if currentServerRun == None and line.startswith("run"):
        currentServerRun = line.split(" ")[1]
    elif line.startswith("Utilization of core 0 is:"):
        utilIsNextLine = True
    elif utilIsNextLine == True:
        utilIsNextLine = False
        util = line
    elif currentServerRun != None and line.startswith("run"):
        results[currentServerRun]["CpuUtilization"] = util
        currentServerRun = line.split(" ")[1]
results[currentServerRun]["CpuUtilization"] = util


In [4]:
paramsFile = open(paramsFilename, "r")

for line in paramsFile.readlines():
    line = line.rstrip()
    splitted = line.split(" ")
    run = splitted[1][:-1]
    sessions = splitted[2].split("=")[1]
    period = splitted[3].split("=")[1]
    i = 0
    for i in range(0, 4):
        id = run + "-" + str(i)
        results[id]["Sessions"] = sessions
        results[id]["Period"] = period
        i = i + 1

In [5]:
parsedResultsFile = open(parsedResultsFilename, "w")

parsedResultsFile.write("Run #, Period, Number of Sessions, Connection Time, Reply Time, CPU Utilization, Network Bandwidth, Network I/O\n")
for key, value in results.items():
    parsedResultsFile.write(key.replace("-", "_") + ",")
    parsedResultsFile.write(value["Period"] + ",")
    parsedResultsFile.write(value["Sessions"] + ",")
    parsedResultsFile.write(value["connectionTime"] + ",")
    parsedResultsFile.write(value["replyTime"] + ",")
    parsedResultsFile.write(value["CpuUtilization"] + ",")
    parsedResultsFile.write(value["bandwidth"] + ",")
    parsedResultsFile.write(value["netIO"] + "\n")

parsedResultsFile.close()


In [18]:
parsedResultsFile = open(parsedResultsFilename, "r")

currentResultsRun = None
arrivalRateSum = 0

responseSum = 0
cpuUtilSum = 0 
networkUtilizationSum = 0

count = 0


def addToValues(p, c, r, u, b, n):
    global arrivalRateSum, responseSum, cpuUtilSum, networkUtilizationSum, count
    arrivalRateSum = arrivalRateSum + (1/float(p))
    
    responseSum = responseSum + float(r) + float(c)
    cpuUtilSum = cpuUtilSum + float(u)
    
    nwu = float(n)/float(b)
    networkUtilizationSum = networkUtilizationSum + nwu
    
    count = count + 1

def clearValues():
    global arrivalRateSum, responseSum, cpuUtilSum, networkUtilizationSum, count
    arrivalRateSum = 0

    responseSum = 0
    cpuUtilSum = 0 
    networkUtilizationSum = 0

    count = 0
    
def calculateAverage(num, count):
    return str(num / count)

def generateAverages():
    global arrivalRateSum, responseSum, cpuUtilSum, networkUtilizationSum, count
    arrivalRateAvg = calculateAverage(arrivalRateSum, count)
    
    responseAvg = calculateAverage(responseSum, count)
    cpuUtilAvg = calculateAverage(cpuUtilSum, count)
    
    networkUtilizationAvg = calculateAverage(networkUtilizationSum, count)
    return {
        "Arrival Rate": arrivalRateAvg,
        "Response Time": responseAvg,
        "Cpu Utilization": cpuUtilAvg,
        "Network Utilization": str(float(networkUtilizationAvg) * 100)
    }
    
finalResults = dict()
for line in parsedResultsFile.readlines():
    line = line.rstrip()
    if line.startswith("Run"):
        continue
        
    run, period, sessions, connection, reply, cpuUtil, bandwidth, network = line.split(",")[0:8]
    resultsRun = run.split("_")[0]
    if currentResultsRun == None:
        currentResultsRun = resultsRun
        addToValues(period, connection, reply, cpuUtil, bandwidth, network)
    elif currentResultsRun == resultsRun:
        addToValues(period, connection, reply, cpuUtil, bandwidth, network)
    elif currentResultsRun != resultsRun:
        finalResults[currentResultsRun] = generateAverages()
        currentResultsRun = resultsRun 
        clearValues()
        addToValues(period, connection, reply, cpuUtil, bandwidth, network)

finalResults[currentResultsRun] = generateAverages()
print(finalResults)

{'0': {'Arrival Rate': '10.0', 'Response Time': '2.5750000000000006', 'Cpu Utilization': '13.7575', 'Network Utilization': '0.2590250472669927'}, '1': {'Arrival Rate': '15.001500150015001', 'Response Time': '2.5500000000000003', 'Cpu Utilization': '16.814999999999998', 'Network Utilization': '0.29331373992896126'}, '2': {'Arrival Rate': '20.0', 'Response Time': '2.85', 'Cpu Utilization': '21.6325', 'Network Utilization': '0.41071744303509733'}, '3': {'Arrival Rate': '25.0', 'Response Time': '3.5750000000000006', 'Cpu Utilization': '25.4', 'Network Utilization': '0.5053854327846237'}, '4': {'Arrival Rate': '30.003000300030003', 'Response Time': '4.075', 'Cpu Utilization': '31.805', 'Network Utilization': '0.6018306332842416'}, '5': {'Arrival Rate': '35.00175008750438', 'Response Time': '5.375', 'Cpu Utilization': '41.302499999999995', 'Network Utilization': '0.7744830684812919'}, '6': {'Arrival Rate': '40.0', 'Response Time': '6.400000000000001', 'Cpu Utilization': '47.47', 'Network Uti

In [21]:
averagedResultsFile = open("AveragedResults.csv", "w")

averagedResultsFile.write("Run #, Arrival Rate, Average Response Time (ms), Average CPU Utilization (%), Average Network Utilization (%)\n")
for key, value in finalResults.items():
    averagedResultsFile.write(key + ",")
    averagedResultsFile.write(value["Arrival Rate"] + ",")
    averagedResultsFile.write(value["Response Time"] + ",")
    averagedResultsFile.write(value["Cpu Utilization"] + ",")
    averagedResultsFile.write(value["Network Utilization"] + "\n")

averagedResultsFile.close()

In [20]:
averagedResultsFile = open("AveragedResults.csv", "r")
averagedResultsFile.read()

'Run #, Arrival Rate, Average Response Time (ms), Average CPU Utilization, Average Network Utilization\n0,10.0,2.5750000000000006,13.7575,0.2590250472669927\n1,15.001500150015001,2.5500000000000003,16.814999999999998,0.29331373992896126\n2,20.0,2.85,21.6325,0.41071744303509733\n3,25.0,3.5750000000000006,25.4,0.5053854327846237\n4,30.003000300030003,4.075,31.805,0.6018306332842416\n5,35.00175008750438,5.375,41.302499999999995,0.7744830684812919\n6,40.0,6.400000000000001,47.47,0.8069292328450609\n7,45.00450045004501,8.7,49.57749999999999,0.8990268431935683\n8,50.0,12.475,51.94500000000001,1.0227677243153557\n9,55.005500550055004,21.849999999999998,55.0025,1.1002814810727293\n10,60.024009603841534,22.325,57.587500000000006,1.1570823529411765\n11,65.01950585175553,23.800000000000004,59.5,1.173761934730258\n12,70.0280112044818,30.950000000000003,62.9225,1.214037208697912\n13,75.01875468867217,37.37500000000001,65.58500000000001,1.2890055011695398\n14,80.0,46.62500000000001,66.11,1.354079528